In [ ]:
%load_ext dotenv
%dotenv

In [ ]:
from pydantic import BaseSettings

class Settings(BaseSettings):
    user_id: int
    password: str

settings = Settings()


In [11]:
from dca.xAPIConnector import APIClient, loginCommand, tradeTransactionCommand
from dca.models import TradeTransInfo

class XTBClient:
    def __init__(self, user_id, password):
        self.client = APIClient()
        self.client.execute(loginCommand(userId=user_id, password=password))
    
    def buy_1_snp_500(self):
        t = TradeTransInfo(cmd=0, customComment='test', expiration=0, offset=0, order=0, price=1_000, sl=0, symbol='CSPX.UK_9', tp=0, type=0, volume=1)
        return str(self.client.execute(tradeTransactionCommand(t)))

client = XTBClient(settings.user_id, settings.password)
client.buy_1_snp_500()

[2022-09-13 13:16:53,154][connect:79] Socket connected
[2022-09-13 13:16:53,157][_waitingSend:93] Sent: b'{"command": "login", "arguments": {"userId": 13794983, "password": "zaq1@WSX", "appName": ""}}'
[2022-09-13 13:16:53,346][_read:112] Received: {'status': True, 'streamSessionId': 'e41f13fffecd98b0-0000441b-0090f1a6-2b4b9a2668ae3131-1e7cf65e'}
[2022-09-13 13:16:53,347][_waitingSend:93] Sent: b'{"command": "tradeTransaction", "arguments": {"tradeTransInfo": {"cmd": 0, "customComment": "test", "expiration": 0, "offset": 0, "order": 0, "price": 1000, "sl": 0, "symbol": "CSPX.UK_9", "tp": 0, "type": 0, "volume": 1}}}'
[2022-09-13 13:16:53,481][_read:112] Received: {'status': True, 'returnData': {'order': 420369030}}


cmd=0 customComment='test' expiration=0 offset=0 order=0 price=Decimal('1000') sl=Decimal('0') symbol='CSPX.UK_9' tp=Decimal('0') type=0 volume=Decimal('1')


"{'status': True, 'returnData': {'order': 420369030}}"